working buoys: use lowercase!! "44003", "44005","44007","44008","44011","44013","44018","44021","44024","44027","44031","44038","44073","44098","iosn3","mdrm1","mism1"

Neracoss Buoys: "44029","44030","44032","44033","44037"

Useless that aren't Neracoss: "44092", "44585", "44003"

In [30]:
#*GOOD*#
"""
put the thingabob here
"""

using CSV, DataFrames, Dates, DateFormats #CSV and DataFrame are neccessary for this function to run

function buoys(ID::String, Startyear::Int, Endyear::Int, Format_Check = false) #Several IDs located in the gulf of maine can be found in the README, Others on the NDBC website #Format_Check is true or false
    #This part is for creating the URL that the standard meteorlogical data is found at on the NDBC Website 
    part1 = "https://www.ndbc.noaa.gov/view_text_file.php?filename="
    part2 = string(ID)
    part3 = "h"
    part5 = ".txt.gz&dir=data/historical/stdmet/"

    #BuoyDF = DataFrame(YY = Float64[],MM = Float64[],DD = Float64[],hh = Float64[],mm = Float64[],WDIR = Float64[], WSPD = Float64[],GST = Float64[],WVHT = Float64[],DPD = Float64[],APD = Float64[], MWD = Float64[], PRES = Float64[],ATMP = Float64[],WTMP = Float64[],DEWP = Float64[],VIS = Float64[],TIDE = Float64[])


    for i in range(Startyear, Endyear) #This loop allows for a range of years to be pulled at the same time, formatted, and saved to your computer.
        part4 = string(i) #The final part of the URL is the year the data was collected on
        URL = part1*part2*part3*part4*part5 

        try #Try statements allow for Startyear and Endyear to be invalid values without causing the code to stop running. Feel free to put whatever wacky years you feel neccessary.
            buoydata = CSV.read(download(URL), DataFrame, delim=" ", ignorerepeated=true,silencewarnings=true,) #In order to get the data off of the NDBC website we make use of the CSV.read function from CSV. 
            #We use Dataframe for the base formating. the data comes delimited by spaces and ignore any repeated spaces. we silence warnings. 
            path = "C:/Users/Cooki/Documents/GitHub/261/PODS/buoy/"*string(ID)*"/"*part4*".csv" #In order to save the data to the computer, the folder must be created seperately by the user
            CSV.write(path,DataFrame(buoydata),header = true) #The data is then saved as a comma seperated dataset at the given path.
        catch e #Any errors that come up when attempting the try statement will result in the following print statement
            println(URL," Does not exist") #If the URL does not exist, an error will display notifying the user that the URL was invalid.
        end

        #Why Format Explanation:
        try 
            path = "C:/Users/Cooki/Documents/GitHub/261/PODS/buoy/"*string(ID)*"/"*part4*".csv" #The same path before is used again in order to format the dataset
            WorkingDF = CSV.read(path, DataFrame, header=1, delim=",", ignorerepeated=false,silencewarnings = true) #The path used before is used again to locate the file and read it in as CSV.

            if typeof(WorkingDF[1,1]) != Int64 #Some data from NDBC has two headers rows instead of 1. this tests if the 2nd row of the year column has a value that is not an integer, which is only true if there are two header rows.
                WorkingDF = CSV.read(path, DataFrame, header=1:2, delim=",", ignorerepeated=false,silencewarnings = true) #The file is reread now accounting for two header rows.
            end
            allowmissing!(WorkingDF)

            #In order to format the dataset, known unwanted header names must be known. Included are several but if more are discovered follow the process in the Markdown cell below on how to account for it.
            #YY
            if (match(r"YYYY|#YY_#yr|YY",string(names(WorkingDF))) != nothing) == true #Match checks to see if possible unwanted header names are present in the list of header names for our dataset.
                #!= nothing means we want the statement to be true when there are any matches at all.
                #Next, individual headers are checked against the list of header names and replaced if are present.
                if issubset(["YYYY"],names(WorkingDF)) == true #issubset will take in a list element(s) and check if it is a subset of a different list. If true means the elements(s) are a subset.
                    rename!(WorkingDF, "YYYY" => "YY") #Once the unwanted header is found, it is renamed to the desired header.
                elseif issubset(["#YY_#yr"],names(WorkingDF)) == true
                    rename!(WorkingDF, :"#YY_#yr" => :YY)
                elseif issubset(["YY"],names(WorkingDF)) == false #If it's found that none of the unwanted headers are present AND the wanted header is not present, it means there is a new unwanted header and warning will be printed.
                    println("STOP, YY!", names(WorkingDF))
                end
            end

             #MM
            if (match(r"(MM_mo|MM)",string(names(WorkingDF))) != nothing) == true
                if issubset(["MM_mo"],names(WorkingDF)) == true
                    rename!(WorkingDF, :MM_mo => :MM)
                elseif issubset(["MM"],names(WorkingDF)) == false 
                    println("STOP, MM!",names(WorkingDF))
                end
            end

            #DD
            if (match(r"(DD_dy|DD)",string(names(WorkingDF))) != nothing) == true
                if issubset(["DD_dy"],names(WorkingDF)) == true
                    rename!(WorkingDF, :DD_dy => :DD)
                elseif issubset(["DD"],names(WorkingDF)) == false 
                    println("STOP, DD!",names(WorkingDF))
                end
            end

            #hh
            if (match(r"(hh_hr|hh)",string(names(WorkingDF))) != nothing) == true
                if issubset(["hh_hr"],names(WorkingDF)) == true
                    rename!(WorkingDF, :hh_hr => :hh)
                elseif issubset(["hh"],names(WorkingDF)) == false 
                    println("STOP, hh!",names(WorkingDF))
                end
            end
        
            #mm
            if (match(r"(mm_mn|mm)",string(names(WorkingDF))) != nothing) == true
                if issubset(["mm_mn"],names(WorkingDF)) == true
                    rename!(WorkingDF, :mm_mn => :mm)

                    #datetime
                    WorkingDF.DATETIME = string.(WorkingDF.YY,"-",WorkingDF.MM,"-",WorkingDF.DD," ",WorkingDF.hh,":",WorkingDF.mm)
                    WorkingDF.DATETIME = DateTime.(WorkingDF.DATETIME,"yyyy-mm-dd HH:MM")

                elseif issubset(["mm"],names(WorkingDF)) == false  
                    println("STOP, mm!",names(WorkingDF))
                end
            #elseif issubset(["mm"],names(WorkingDF)) == true
                #datetime
                WorkingDF.DATETIME = string.(WorkingDF.YY,"-",WorkingDF.MM,"-",WorkingDF.DD," ",WorkingDF.hh,":",WorkingDF.mm)
                WorkingDF.DATETIME = DateTime.(WorkingDF.DATETIME,"yyyy-mm-dd HH:MM")
            else
                insertcols!(WorkingDF,5,:mm => missing, makeunique = true) #In the case of mm and TIDE, most of the NDBC data sets do not include one or the other for a number of years. This interserts a column for the sake of formating.

                #datetime
                WorkingDF.DATETIME = string.(WorkingDF.YY,"-",WorkingDF.MM,"-",WorkingDF.DD," ",WorkingDF.hh)
                WorkingDF.DATETIME = DateTime.(WorkingDF.DATETIME,"yyyy-mm-dd HH")
                
            end
        

            #WDIR
            if (match(r"(WD|WDIR_degT|WDIR)",string(names(WorkingDF))) != nothing) == true
                if issubset(["WD"],names(WorkingDF)) == true
                    rename!(WorkingDF, :WD => :WDIR)
                elseif issubset(["WDIR_degT"],names(WorkingDF)) == true
                    rename!(WorkingDF, :WDIR_degT => :WDIR)
                elseif issubset(["WDIR"],names(WorkingDF)) == false 
                    println("STOP, WDIR!",names(WorkingDF))
                end
            end
        
            #WSPD
            if (match(r"(WSPD|WSPD_m/s)",string(names(WorkingDF))) != nothing) == true
                if issubset(["WSPD_m/s"],names(WorkingDF)) == true
                    rename!(WorkingDF, "WSPD_m/s" => :WSPD)
                elseif issubset(["WSPD"],names(WorkingDF)) == false 
                    println("STOP, WSPD!",names(WorkingDF))
                end
            end
        
            #GST
            if (match(r"(GST|GST_m/s)",string(names(WorkingDF))) != nothing) == true
                if issubset(["GST_m/s"],names(WorkingDF)) == true
                    rename!(WorkingDF, "GST_m/s" => :GST)
                elseif issubset(["GST"],names(WorkingDF)) == false 
                    println("STOP, GST!",names(WorkingDF))
                end
            end
        
            #WVHT
            if (match(r"(WVHT|WVHT_m)",string(names(WorkingDF))) != nothing) == true
                if issubset(["WVHT_m"],names(WorkingDF)) == true
                    rename!(WorkingDF, :WVHT_m => :WVHT)
                elseif issubset(["WVHT"],names(WorkingDF)) == false 
                    println("STOP, WVHT!",names(WorkingDF))
                end
            end
        
            #DPD
            if (match(r"(DPD|DPD_sec)",string(names(WorkingDF))) != nothing) == true
                if issubset(["DPD_sec"],names(WorkingDF)) == true
                    rename!(WorkingDF, :DPD_sec => :DPD)
                elseif issubset(["DPD"],names(WorkingDF)) == false 
                    println("STOP, DPD!",names(WorkingDF))
                end
            end
        
            #APD
            if (match(r"(APD|APD_sec)",string(names(WorkingDF))) != nothing) == true
                if issubset(["APD_sec"],names(WorkingDF)) == true
                    rename!(WorkingDF, :APD_sec => :APD)
                elseif issubset(["APD"],names(WorkingDF)) == false 
                    println("STOP, APD!",names(WorkingDF))
                end
            end
        
            #MWD
            if (match(r"(MWD|MWD_degT|MWD_deg)",string(names(WorkingDF))) != nothing) == true
                if issubset(["MWD_degT"],names(WorkingDF)) == true
                    rename!(WorkingDF, :MWD_degT => :MWD)
                elseif issubset(["MWD_deg"],names(WorkingDF)) == true
                    rename!(WorkingDF, :MWD_deg => :MWD) 
                elseif issubset(["MWD"],names(WorkingDF)) == false 
                    println("STOP, MWD!",names(WorkingDF))
                end
            end
        
            #PRES
            if (match(r"(BAR|PRES|PRES_hPa)",string(names(WorkingDF))) != nothing) == true
                if issubset(["BAR"],names(WorkingDF)) == true
                    rename!(WorkingDF, :BAR => :PRES)
                elseif issubset(["PRES_hPa"],names(WorkingDF)) == true
                    rename!(WorkingDF, :PRES_hPa => :PRES)
                elseif issubset(["PRES"],names(WorkingDF)) == false 
                    println("STOP, PRES!",names(WorkingDF))
                end
            end
        
            #ATMP
            if (match(r"(ATMP|ATMP_degC)",string(names(WorkingDF))) != nothing) == true
                if issubset(["ATMP_degC"],names(WorkingDF)) == true
                    rename!(WorkingDF, :ATMP_degC => :ATMP)
                elseif issubset(["ATMP"],names(WorkingDF)) == false 
                    println("STOP, ATMP!",names(WorkingDF))
                end
            end
        
            #WTMP
            if (match(r"(WTMP|WTMP_degC)",string(names(WorkingDF))) != nothing) == true
                if issubset(["WTMP_degC"],names(WorkingDF)) == true
                    rename!(WorkingDF, :WTMP_degC => :WTMP)
                elseif issubset(["WTMP"],names(WorkingDF)) == false 
                    println("STOP, WTMP!",names(WorkingDF))
                end
            end
        
            #DEWP
            if (match(r"(DEWP|DEWP_degC)",string(names(WorkingDF))) != nothing) == true
                if issubset(["DEWP_degC"],names(WorkingDF)) == true
                    rename!(WorkingDF, :DEWP_degC => :DEWP)
                elseif issubset(["DEWP"],names(WorkingDF)) == false  
                    println("STOP, DEWP!",names(WorkingDF))
                end
            end
        
            #VIS
            if (match(r"(VIS|VIS_mi|VIS_nmi)",string(names(WorkingDF))) != nothing) == true
                if issubset(["VIS_mi"],names(WorkingDF)) == true
                    rename!(WorkingDF, :VIS_mi => :VIS)
                elseif issubset(["VIS_nmi"],names(WorkingDF)) == true
                    rename!(WorkingDF, :VIS_nmi => :VIS)
                elseif issubset(["VIS"],names(WorkingDF)) == false 
                    println("STOP, VIS!",names(WorkingDF))
                end
            end
        
            #TIDE
            if (match(r"(TIDE|TIDE_ft)",string(names(WorkingDF))) != nothing) == true
                if issubset(["TIDE_ft"],names(WorkingDF)) == true
                    rename!(WorkingDF, :TIDE_ft => :TIDE)
                elseif issubset(["TIDE"],names(WorkingDF)) == false 
                    println("STOP, TIDE!",names(WorkingDF))
                end
            else
                insertcols!(WorkingDF,18,:TIDE => missing, makeunique = true)
            end
        
            #A number of YY values were collected as two digit values instead of the current four digit values. For datetime manipulation this must be corrected.
            for j in range(i-1,i+1) #Some Buoys have year values from the year before or after, this loop corrects those as well.
                Old_YY = parse(Int64,string(j)[end-1:end]) #the last two values of the integer 'j' are made into a string and turned into an integer
                replace!(WorkingDF.YY, Old_YY => j) #The reduced version of 'j', 'Old_YY', is replaced in any location in the YY column by 'j'
            end

            #Julian Datetime
            WorkingDF.JULIAN_DATE = Dates.datetime2julian.(WorkingDF.DATETIME)
        
            #Select! will reorder the columns
            select!(WorkingDF, [:YY,:MM,:DD,:hh,:mm,:DATETIME,:JULIAN_DATE,:WDIR,:WSPD,:GST,:WVHT,:DPD,:APD,:MWD,:PRES,:ATMP,:WTMP,:DEWP,:VIS,:TIDE])

            #The option of format check turns on and off a print statement which displays the year and header names once complete.
            if Format_Check == true 
                println(i,names(WorkingDF))
            else 
                nothing
            end

            CSV.write(path,DataFrame(WorkingDF),header = true) #Once formating is complete, the file is rewritten.

        catch e #Any errors found when attempting the try statement are printed below with the year
            println("Format Error: ", i, e)
        end
    end
    return println("Done") #Because this function does not return anything else, a print statement is included to show that process was completed.
end

buoys (generic function with 2 methods)

In [ ]:
buoys("44007",1985,2020,true)